In [1]:
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
sys.path.append(project_root)

from util.config_loader import load_config_api
from util.api_client import ApiClient

# Chat API

In [2]:
cfg = load_config_api("chat", path="../config.ini")
api = ApiClient(cfg)

Base URL set to: https://b65cb04e9d05.ngrok.app


In [3]:
body, status, headers = api.post('/ask', {'question': 'Who are you?'})
print("Status:", status)
print("Headers:", headers)
print("Body:", body)

Status: 200
Headers: {'Content-Length': '112', 'Content-Type': 'application/json', 'Date': 'Tue, 04 Nov 2025 06:02:07 GMT', 'Server': 'uvicorn', 'Connection': 'close'}
Body: {'answer': 'I am Gemma, an open-weights AI assistant. I am a large language model trained by Google DeepMind.\n'}


# Embedding API

In [4]:
cfg = load_config_api("embedding", path="../config.ini")
api = ApiClient(cfg)

Base URL set to: https://civilly-uncognizable-nilda.ngrok-free.dev


In [ ]:
body, status, headers = api.post('/ask', {'question': 'Who are you?'})
print("Status:", status)
print("Headers:", headers)
print("Body:", body)